# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import astropy.units as u
import jax.numpy as jnp
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astroquery.jplhorizons import Horizons

from jorbit.mpchecker import mpchecker
from jorbit.utils.horizons import horizons_bulk_astrometry_query


In [2]:
obj = Horizons(id="Ceres", location="500@399", epochs=[Time("2021-01-01").utc.jd])
eph = obj.ephemerides(extra_precision=True, quantities="1,9")
coord = SkyCoord(ra=eph["RA"][0], dec=eph["DEC"][0], unit=(u.deg, u.deg))
coord

<SkyCoord (ICRS): (ra, dec) in deg
    (346.8268329, -15.83129517)>

In [3]:
mpchecker(
    coordinate=coord,
    time=Time("2021-01-01 00:05"),
    radius=10*u.arcmin,
    extra_precision=True,
    observer="geocentric",
    extra_precision_gravity="newtonian solar system",
    chunk_coefficients=None,
)

ValueError: Of the objects found nearby the target, ['00001'] are massive perturbers that are included in jorbit ephemeris calculations by default. Consequently, jorbit cannot actually integrate this target itself since doing so would produce infinite accelerations. These will be dropped from consideration here: use the Ephemeris module, or an external service like JPL Horizons, to get the ephemeris of these objects.

In [3]:
from jorbit.astrometry.sky_projection import sky_sep
from jorbit.data.constants import JORBIT_EPHEM_URL_BASE
from jorbit.mpchecker.parse_jorbit_ephem import (
    extra_precision_calcs,
    get_chunk_index,
    get_relevant_mpcorb,
    multiple_states,
    setup_checks,
)
from jorbit.utils.cache import download_file_wrapper

coordinate=coord
time=Time("2021-01-01 00:05")
radius=10*u.arcmin
extra_precision=True
observer="geocentric"
extra_precision_gravity="newtonian solar system"
chunk_coefficients=None


if observer != "geocentric":
    extra_precision = True

coordinate, radius, t0, tf, chunk_size, names = setup_checks(
    coordinate, time, radius
)

index, offset = get_chunk_index(time.tdb.jd, t0, tf, chunk_size)

# figure out what chunk you're in
if chunk_coefficients is None:
    if jnp.sign(index) == -1:
        file_name = JORBIT_EPHEM_URL_BASE + f"chebyshev_coeffs_rev_{-index:03d}.npy"
    else:
        file_name = JORBIT_EPHEM_URL_BASE + f"chebyshev_coeffs_fwd_{index:03d}.npy"
    file_name = download_file_wrapper(file_name)
    coefficients = jnp.load(file_name)

# get the ra and dec of every minor planet (!)
ras, decs = multiple_states(coefficients, offset, t0, chunk_size)

# get the separation in arcsec
separations = jax.vmap(sky_sep, in_axes=(0, 0, None, None))(
    ras, decs, coordinate.ra.rad, coordinate.dec.rad
)

# filter down to just those within the radius
mask = separations < radius
names = names[mask]
ras = ras[mask]
decs = decs[mask]
separations = separations[mask]

relevant_mpcorb = get_relevant_mpcorb(mask)

if extra_precision:
    # coords, seps, coord_table, mags, mag_table, total_mags = extra_precision_calcs(
    q = extra_precision_calcs(
        asteroid_flags=mask,
        times=time,
        radius=radius,
        observer=observer,
        coordinate=coordinate,
        gravity=extra_precision_gravity,
        relevant_mpcorb=relevant_mpcorb,
    )

/var/folders/mj/qxz5chg95r53_2nlv9f86qhm0000gn/T/ipykernel_7015/257075223.py:58: UserWarning: Of the objects found nearby the target, ['00001'] are massive perturbers that are included in jorbit ephemeris calculations by default. Consequently, jorbit cannot actually integrate this target itself since doing so would produce infinite accelerations. It will query JPL Horizons for this object, but all queries must now be limited to fewer than 10k unique timesteps.
  q = extra_precision_calcs(


12
11


In [4]:
relevant_mpcorb

Unpacked Name,Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,last obs
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""(1) Ceres""","""00001""","""3.34""","""0.15""","""K2555""","""188.70269""","""73.27343""","""80.25221""","""10.58780""","""0.0794013""","""0.21424651""","""2.7660512""","""0""","""E2024-V47""","""7330""","""125""","""1801-2024""","""0.80""","""M-v""","""30k""","""MPCLINUX""","""4000""","""20241101"""
"""(6655) Nagahama""","""06655""","""11.69""","""0.15""","""K2555""","""258.85899""","""333.23929""","""74.95262""","""10.75076""","""0.0775850""","""0.19000839""","""2.9965454""","""0""","""E2024-TQ2""","""4684""","""27""","""1964-2024""","""0.62""","""M-v""","""3Ek""","""MPCLINUX""","""0000""","""20241011"""
"""(7163) Barenboim""","""07163""","""15.18""","""0.15""","""K2555""","""16.70765""","""310.69456""","""154.50635""","""20.76360""","""0.1879659""","""0.28667753""","""2.2779295""","""0""","""E2025-D28""","""4250""","""22""","""1984-2025""","""0.56""","""M-v""","""3Ek""","""MPCLINUX""","""0007""","""20250218"""
"""2014 DE160""","""K14DG0E""","""18.3""","""0.15""","""K2555""","""273.97079""","""350.89104""","""126.24699""","""12.91669""","""0.1500130""","""0.23602418""","""2.5931744""","""3""","""MPO519013""","""26""","""2""","""2014-2018""","""0.25""","""M-v""","""38h""","""Veres""","""0000""","""20180212"""
"""2015 LV8""","""K15L08V""","""18.36""","""0.15""","""K2555""","""74.37435""","""203.95321""","""52.26685""","""13.39119""","""0.1829183""","""0.22210835""","""2.7003877""","""2""","""MPO687275""","""48""","""2""","""2015-2020""","""0.64""","""M-v""","""3Ek""","""MPCLINUX""","""0000""","""20200814"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""(334958) 2004 CW111""","""X4958""","""16.38""","""0.15""","""K2555""","""150.53994""","""60.24083""","""121.79330""","""12.49960""","""0.0627954""","""0.21047320""","""2.7990125""","""0""","""E2024-C06""","""346""","""13""","""2002-2023""","""0.75""","""M-v""","""3Ek""","""MPCLINUX""","""0000""","""20230513"""
"""(353943) 1998 TU29""","""Z3943""","""17.11""","""0.15""","""K2555""","""341.58496""","""242.80038""","""146.28206""","""14.91310""","""0.3482082""","""0.22067204""","""2.7120926""","""0""","""MPO843755""","""266""","""12""","""1998-2024""","""0.87""","""M-v""","""3Ek""","""MPCLINUX""","""0000""","""20240627"""
"""(596757) 2006 CN21""","""x6757""","""16.51""","""0.15""","""K2555""","""264.71629""","""244.52714""","""144.32504""","""17.33681""","""0.1853692""","""0.17818857""","""3.1276357""","""0""","""E2024-UN8""","""110""","""8""","""2006-2024""","""0.86""","""M-v""","""3Ek""","""MPCLINUX""","""0000""","""20240504"""


In [5]:
q

In [6]:
# array(['00001'], dtype='<U7')

In [ ]:
obj = Horizons(id="274301", location="500@399", epochs=[Time("2021-01-01").utc.jd])
eph = obj.ephemerides(extra_precision=True, quantities="1,9")
eph

targetname,datetime_str,datetime_jd,H,G,solar_presence,lunar_presence,RA,DEC,V,surfbright
---,---,d,mag,---,---,---,deg,deg,mag,mag / arcsec2
str28,str24,float64,float64,float64,str1,str1,float64,float64,float64,int64
274301 Wikipedia (2008 QH24),2021-Jan-01 00:00:00.000,2459215.5,16.98,0.15,,,119.98362688,10.559774616,20.857,--


In [ ]:
horizons_bulk_astrometry_query(
    target="274301",
    center="500@399",
    times=Time("2021-01-01"),
)

,JD_UTC,RA,DEC,V,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma
0,2459215.5,119.983627,10.559775,20.857,0.093,0.076,0.094,0.076,-6.254,0.022256


In [4]:
from jorbit.mpchecker.parse_jorbit_ephem import load_mpcorb

m = load_mpcorb()

In [5]:
m

Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,Unpacked Name,last obs
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""00001""","""3.34""","""0.15""","""K2555""","""188.70269""","""73.27343""","""80.25221""","""10.58780""","""0.0794013""","""0.21424651""","""2.7660512""","""0""","""E2024-V47""","""7330""","""125""","""1801-2024""","""0.80""","""M-v""","""30k""","""MPCLINUX""","""4000""","""(1) Ceres""","""20241101"""
"""00002""","""4.11""","""0.15""","""K2555""","""168.79870""","""310.91037""","""172.89530""","""34.92402""","""0.2305404""","""0.21376611""","""2.7701937""","""0""","""E2024-V16""","""8964""","""123""","""1804-2024""","""0.75""","""M-c""","""28k""","""Pan""","""4000""","""(2) Pallas""","""20240927"""
"""00003""","""5.18""","""0.15""","""K2555""","""172.45499""","""247.85972""","""169.82994""","""12.98651""","""0.2559472""","""0.22582594""","""2.6706696""","""0""","""E2024-V16""","""7523""","""117""","""1804-2024""","""0.83""","""M-v""","""3Ek""","""Pan""","""4000""","""(3) Juno""","""20240607"""
"""00004""","""3.25""","""0.15""","""K2555""","""332.45069""","""151.58335""","""103.70297""","""7.14394""","""0.0901071""","""0.27161293""","""2.3613975""","""0""","""E2024-V16""","""7647""","""111""","""1821-2024""","""0.82""","""M-p""","""18k""","""Pan""","""4000""","""(4) Vesta""","""20240306"""
"""00005""","""6.99""","""0.15""","""K2555""","""86.24723""","""359.31775""","""141.45353""","""5.35887""","""0.1874407""","""0.23828278""","""2.5767618""","""0""","""E2025-CG9""","""3184""","""90""","""1845-2025""","""0.86""","""M-v""","""3Ek""","""MPCLINUX""","""4000""","""(5) Astraea""","""20250213"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""K21A03O""","""19.8""","""0.15""","""K2116""","""84.30053""","""28.49179""","""348.07893""","""17.65767""","""0.1489933""","""0.37146042""","""1.9165832""",null,"""MPO600176""","""10""","""1""","""2 days""","""0.20""",null,null,"""Bell""","""2006""","""2021 AO3""","""20210110"""
"""K21C09X""","""21.9""","""0.15""","""K211Q""","""353.12519""","""128.19322""","""23.78627""","""9.07555""","""0.4556361""","""0.23694889""","""2.5864233""",null,"""E2021-D18""","""10""","""1""","""4 days""","""0.07""",null,null,"""Bell""","""2005""","""2021 CX9""","""20210216"""
"""K21C10S""","""20.6""","""0.15""","""K212F""","""164.66648""","""223.65372""","""119.89392""","""29.38693""","""0.0526282""","""0.44236564""","""1.7058799""",null,"""E2021-DA1""","""9""","""1""","""11 days""","""0.07""",null,null,"""Bell""","""2005""","""2021 CS10""","""20210223"""


In [11]:
import polars as pl

m = load_mpcorb()
for col in [
 'H',
 'G',
 'M',
 'Peri',
 'Node',
 'Incl.',
 'e',
 'n',
 'a',
 '#Obs',
 '#Opp',
 'rms']:
    m = m.with_columns(pl.col(col).cast(pl.Float64))
m

Packed designation,H,G,Epoch,M,Peri,Node,Incl.,e,n,a,U,Reference,#Obs,#Opp,Arc,rms,Coarse Perts,Precise Perts,Computer,Flags,Unpacked Name,last obs
str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,str,str,f64,f64,str,f64,str,str,str,str,str,str
"""00001""",3.34,0.15,"""K2555""",188.70269,73.27343,80.25221,10.5878,0.0794013,0.214247,2.7660512,"""0""","""E2024-V47""",7330.0,125.0,"""1801-2024""",0.8,"""M-v""","""30k""","""MPCLINUX""","""4000""","""(1) Ceres""","""20241101"""
"""00002""",4.11,0.15,"""K2555""",168.7987,310.91037,172.8953,34.92402,0.2305404,0.213766,2.7701937,"""0""","""E2024-V16""",8964.0,123.0,"""1804-2024""",0.75,"""M-c""","""28k""","""Pan""","""4000""","""(2) Pallas""","""20240927"""
"""00003""",5.18,0.15,"""K2555""",172.45499,247.85972,169.82994,12.98651,0.2559472,0.225826,2.6706696,"""0""","""E2024-V16""",7523.0,117.0,"""1804-2024""",0.83,"""M-v""","""3Ek""","""Pan""","""4000""","""(3) Juno""","""20240607"""
"""00004""",3.25,0.15,"""K2555""",332.45069,151.58335,103.70297,7.14394,0.0901071,0.271613,2.3613975,"""0""","""E2024-V16""",7647.0,111.0,"""1821-2024""",0.82,"""M-p""","""18k""","""Pan""","""4000""","""(4) Vesta""","""20240306"""
"""00005""",6.99,0.15,"""K2555""",86.24723,359.31775,141.45353,5.35887,0.1874407,0.238283,2.5767618,"""0""","""E2025-CG9""",3184.0,90.0,"""1845-2025""",0.86,"""M-v""","""3Ek""","""MPCLINUX""","""4000""","""(5) Astraea""","""20250213"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""K21A03O""",19.8,0.15,"""K2116""",84.30053,28.49179,348.07893,17.65767,0.1489933,0.37146,1.9165832,null,"""MPO600176""",10.0,1.0,"""2 days""",0.2,null,null,"""Bell""","""2006""","""2021 AO3""","""20210110"""
"""K21C09X""",21.9,0.15,"""K211Q""",353.12519,128.19322,23.78627,9.07555,0.4556361,0.236949,2.5864233,null,"""E2021-D18""",10.0,1.0,"""4 days""",0.07,null,null,"""Bell""","""2005""","""2021 CX9""","""20210216"""
"""K21C10S""",20.6,0.15,"""K212F""",164.66648,223.65372,119.89392,29.38693,0.0526282,0.442366,1.7058799,null,"""E2021-DA1""",9.0,1.0,"""11 days""",0.07,null,null,"""Bell""","""2005""","""2021 CS10""","""20210223"""


In [10]:
m.columns

['Packed designation',
 'H',
 'G',
 'Epoch',
 'M',
 'Peri',
 'Node',
 'Incl.',
 'e',
 'n',
 'a',
 'U',
 'Reference',
 '#Obs',
 '#Opp',
 'Arc',
 'rms',
 'Coarse Perts',
 'Precise Perts',
 'Computer',
 'Flags',
 'Unpacked Name',
 'last obs']

In [8]:
pl.__version__

'1.22.0'